In [1]:
# Dependencies
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, desc
from flask import Flask, jsonify, render_template, request
import csv

In [2]:
# Connect to sqlite database
engine = create_engine("sqlite:///belly_button_biodiversity.sqlite")
Base = automap_base()
Base.prepare(engine, reflect=True)
session = Session(engine)

In [3]:
# Storing tables
Otu = Base.classes.otu
Samples = Base.classes.samples
Samples_Metadata = Base.classes.samples_metadata

In [16]:
def names():

    # Empty list for sample names
    sample_ids = []
    
    # Grab metadata table
    results = session.query(Samples_Metadata.SAMPLEID)

    # Loop through query & grab ids
    for result in results:
        sample_ids.append("BB_" + str(result[0]))

    return jsonify(sample_ids)


In [17]:
def otu():

    # Empty list for descriptions
    otu_desc = []
    
    # Grab otu table
    results = session.query(Otu.lowest_taxonomic_unit_found)

    # Loop through query & grab descriptions
    for result in results:
        otu_desc.append(result[0])

    return jsonify(otu_desc)


In [18]:
def metadata(sample):
    
    # Grab input
    sample_id = int(sample.split("_")[1])

    # Empty dictionary for data
    sample_metadata = {}

    # Grab metadata table
    samples = session.query(Samples_Metadata)

    # Loop through query & grab info
    for info in samples:
        if (sample_id == info.SAMPLEID):
            sample_metadata["AGE"] = info.AGE
            sample_metadata["BBTYPE"] = info.BBTYPE
            sample_metadata["ETHNICITY"] = info.ETHNICITY
            sample_metadata["GENDER"] = info.GENDER
            sample_metadata["LOCATION"] = info.LOCATION
            sample_metadata["SAMPLEID"] = info.SAMPLEID

    return jsonify(sample_metadata)

In [19]:
def wfreq(sample):

    # Grab input
    sample_id = int(sample.split("_")[1])

    # Grab metadata table
    results = session.query(Samples_Metadata)

    # Loop through and grab wfreq
    for result in results:
        if (sample_id == result.SAMPLEID):
            wfreq = result.WFREQ

    return jsonify(wfreq)

In [20]:
def samples(sample):

    # Create sample query
    sample_query = "Samples." + sample

    # Create empty dictionary & lists
    samples_info = {}
    otu_ids = []
    sample_values = []

    # Grab info
    results = session.query(Samples.otu_id, sample_query).order_by(desc(sample_query))

    # Loop through & append
    for result in results:
        otu_ids.append(result[0])
        sample_values.append(result[1])

    # Add to dictionary
    samples_info = {
        "otu_ids": otu_ids,
        "sample_values": sample_values
    }

    return jsonify(samples_info)